# 演習11

## 条件

売り手は,$m$ 個の財を出品し,買い手は,財1個あたりの評価額 $v_i$ を持つ。各買い手は,m 個までの財を需要する。

買い手 $i$ の財1個あたりの入札額を $b_i$ としたとき,財の割当ては次の最適化問題の最適解として導出される。

最適解を導出する際はDPを使用する。

$$
\begin{aligned}
\text{最大化}\quad & \sum_{i \in N} b_i x_i \\
\text{条件}\quad &
\begin{cases}
\displaystyle \sum_{i \in N} x_i \leq m, \\
x \geq 0
\end{cases}
\end{aligned}
\tag{1}
$$

また、財の割り当ては下記で示されるVCGメカニズムに従う。

全員が参加した時のオークションの割り当てを $\vec{x}^{*}$ 、特定の買い手 $j$ を除いたオークションの割り当てを、$\vec{x}^{-j}$ とする。
このとき買い手 $j$ の支払額 $p_j$ は下記のように計算される。

$$
p_j = 
\sum_{i \in N \setminus \{j\}} b_i x_i^{-j}
-
\left(
\sum_{i \in N} b_i x_i^* - b_j x_j^*
\right)
$$





In [9]:
from typing import List

class DPBasedVCGMechanism:
    n_bidders: int
    n_items: int
    bids: List[List[int]]  # 2D valuation matrix [bidder][item]

    def __init__(self):
        pass

# 演習12

# 演習13

# 演習14

# 演習15

クラスHngarianとして実装する

```Python
class Hungarian:
    def __init__(self,X):

```

## 入力X


$$
\mathbf{X}
=
\begin{bmatrix}
x_{1,1} & x_{1,2} & \cdots & x_{1,N} \\
x_{2,1} & x_{2,2} & \cdots & x_{2,N} \\
\vdots  & \vdots  & \ddots & \vdots  \\
x_{N,1} & x_{N,2} & \cdots & x_{N,N}
\end{bmatrix},
\qquad 1 \le i \le N,\; 1 \le j \le N
$$


## step1

## step2

クラスHangarian内にてdef _step(self)として定義する。

## step3

クラスHangarian内にてdef _step3(self)として定義する。

## step4


In [ ]:
from typing import List, Tuple
class Hungarian:
    """
    Hungarian algorithm for assignment problems.
    Attributes:
        X: 3D list representing the cost matrix.
    
    """
    N: int = 0
    input_tensor: List[List[int]] = [[]]
    output : List[int]  = []
    
    def __init__(self,X):
        self.input_tensor = X
        if len(X) != len(X[0]):
            raise ValueError("Input matrix must be square.")
        else:
            self.N = len(X)
        self._step1()
        print(f"After Step 1: {self.input_tensor}")
        while 1:
            if self._step2():
                break
            self._step3()
        self._step3()
        self._step4()
    def _is_include_zero(self,arr):
        for v in arr:
            if v == 0:
                return True
        return False
    def _step1(self):
        for i in range(self.N):
            min_value = min(self.input_tensor[i])
            for j in range(self.N):
                self.input_tensor[i][j] -= min_value
        for j in range(self.N):
            col = [self.input_tensor[i][j] for i in range(self.N)]
            min_value = min(col)
            for i in range(self.N):
                self.input_tensor[i][j] -= min_value
    def _step2(self):
        is_completed = True 
        for i in range(self.N):
            if 0 not in self.input_tensor[i]:
                is_completed = False
        return is_completed
    def _step3(self):
        records: List[Tuple[str, int, int]] = []
        tmp_tensor = self.input_tensor.copy()
        for i in range(self.N):
            num_zeros = tmp_tensor[i].count(0)
            records.append(('row', num_zeros, i))   
        for j in range(self.N):
            col = [tmp_tensor[i][j] for i in range(self.N)]
            num_zeros = col.count(0)
            records.append(('col', num_zeros, j))
        records.sort(key=lambda x: x[1])

    def _step4(self):
        # Placeholder for step 4 of the Hungarian algorithm
        pass
test_tensor = [
    [7, 8, 5, 6],
    [6, 5, 9, 10],
    [4, 1, 10, 7],
    [3, 4, 6, 5]
]

Hungarian(test_tensor)

After Step 1: [[2, 3, 0, 0], [1, 0, 4, 4], [3, 0, 9, 5], [0, 1, 3, 1]]


# 演習16

ハンガリー法を用いて $(AP)$、$(AP)^{-i}$ を計算する


In [5]:
class HungarianVCGAllocator:
    n_bidders: int
    n_items: int
    values: List[List[int]]  # values[i][j] in [10, 100]
    ans_allocation: List[int] = []
    ans_payment: List[int] = []
    def __init__(self, n_bidders: int, n_items: int, values: List[List[int]]):
        self.n_cols = len(values[0])
        self.n_rows = len(values)
        if self.n_cols != n_items or self.n_rows != n_bidders:
            raise ValueError("Dimension of values does not match n_bidders and n_items.")
        self.n_bidders = n_bidders
        self.n_items = n_items
        self.values = values
        self.Vmax = self.to_maximization_matrix()
    def to_maximization_matrix(self) -> List[List[int]]:
        max_value = max(max(row) for row in self.values)
        return [[max_value - self.values[i][j] for j in range(self.n_items)] for i in range(self.n_rows)]   

sample_n_bidders = 3
sample_n_items = 3
sample_tensor = [
    [10, 20, 30],
    [40, 50, 60],
    [70, 80, 90]
]
test_allocator = HungarianVCGAllocator(sample_n_bidders, sample_n_items, sample_tensor)
print(test_allocator.Vmax)

[[80, 70, 60], [50, 40, 30], [20, 10, 0]]
